<a href="https://colab.research.google.com/github/Witcape/3D_Printing/blob/main/INCEPTION_NORMAL_FINAL_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_dir = "/content/drive/MyDrive/Final_Final/Train"
test_dir = "/content/drive/MyDrive/Final_Final/Test"

# List all image paths and corresponding labels
image_paths = []
labels = []

for class_dir in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_dir)

    if not os.path.isdir(class_path):
        continue

    for img in os.listdir(class_path):
        image_paths.append(os.path.join(class_path, img))
        labels.append(class_dir)  # Use the directory name as the label

# Split into train and validation
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)


In [4]:
unique_labels = set(labels)
unique_labels
import pandas as pd

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
# Create DataFrame for training data
train_df = pd.DataFrame({
    "filepath": train_paths,
    "class": train_labels
})

# Create DataFrame for validation data
val_df = pd.DataFrame({
    "filepath": val_paths,
    "class": val_labels
})



val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepath",
    y_col="class",
    target_size=(299, 299),  # Image size for InceptionV3
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="filepath",
    y_col="class",
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

Found 693 validated image filenames belonging to 3 classes.
Found 173 validated image filenames belonging to 3 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 2 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  warnings.warn(


In [6]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [7]:
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling
x = Dense(1024, activation='relu')(x)  # Fully connected layer
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)  # Output layer

model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
history_fine = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


22/22 ━━━━━━━━━━━━━━━━━━━━ 334s 12s/step - accuracy: 0.4448 - loss: 2.0837 - val_accuracy: 0.7919 - val_loss: 0.4886
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 76s 675ms/step - accuracy: 0.8191 - loss: 0.4775 - val_accuracy: 0.8439 - val_loss: 0.3725
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 20s 702ms/step - accuracy: 0.8676 - loss: 0.3449 - val_accuracy: 0.8728 - val_loss: 0.3128
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 22s 696ms/step - accuracy: 0.8840 - loss: 0.3271 - val_accuracy: 0.8497 - val_loss: 0.3903
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 40s 655ms/step - accuracy: 0.9128 - loss: 0.2649 - val_accuracy: 0.8555 - val_loss: 0.3678
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 22s 765ms/step - accuracy: 0.9451 - loss: 0.1601 - val_accuracy: 0.9191 - val_loss: 0.2420
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 40s 710ms/step - accuracy: 0.9371 - loss: 0.1815 - val_accuracy: 0.9133 - val_loss: 0.2245
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 41s 692ms/step - accuracy: 0.9486 - loss: 0.1516 - val_accuracy: 0.8960

In [11]:
for layer in base_model.layers[-50:]:
    layer.trainable = True

In [12]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
history_fine = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step - accuracy: 0.8909 - loss: 0.2884 - val_accuracy: 0.8150 - val_loss: 0.5668
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 50s 706ms/step - accuracy: 0.9412 - loss: 0.1297 - val_accuracy: 0.8555 - val_loss: 0.3995
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 41s 683ms/step - accuracy: 0.9794 - loss: 0.0704 - val_accuracy: 0.9538 - val_loss: 0.1600
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 41s 698ms/step - accuracy: 0.9851 - loss: 0.0460 - val_accuracy: 0.9595 - val_loss: 0.1314
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 713ms/step - accuracy: 0.9814 - loss: 0.0423 - val_accuracy: 0.9595 - val_loss: 0.1437
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 22s 701ms/step - accuracy: 0.9867 - loss: 0.0296 - val_accuracy: 0.9595 - val_loss: 0.1370
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 41s 696ms/step - accuracy: 0.9976 - loss: 0.0143 - val_accuracy: 0.9595 - val_loss: 0.1046
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 41s 708ms/step - accuracy: 0.9966 - loss: 0.0152 - val_accurac

In [14]:
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")

Found 387 images belonging to 3 classes.
13/13 ━━━━━━━━━━━━━━━━━━━━ 165s 14s/step - accuracy: 0.9266 - loss: 0.2281
Test Accuracy: 0.90
